In [3]:
import math
import statistics
import random
import utils
from itertools import combinations

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import csv

In [4]:
_pretty_suits = ["♥", "♦", "♠", "♣"]
_boring_suits = ["H", "D", "S", "C"]

_values = list(range(2, 10))
_values.extend(["T", "J", "Q", "K", "A"])

pretty_deck = []
deck = []
for i in range(len(_pretty_suits)):
    for value in _values:
        pretty_deck.append(f"{value}{_pretty_suits[i]}")
        deck.append(f"{value}{_boring_suits[i]}")

In [5]:
print(len(deck))
print(deck[0:14])

52
['2H', '3H', '4H', '5H', '6H', '7H', '8H', '9H', 'TH', 'JH', 'QH', 'KH', 'AH', '2D']


In [6]:
starting_hand_size = 6
hand_size = 4

In [7]:
# 52c4 * 48
# where 48 is the remaining cards after you get your hand
unique_hands = math.comb(len(deck), hand_size) * (len(deck) - hand_size)
print(f"{unique_hands=:,}")

# 6 cards dealt to you, you have to discard 2
unique_hands_dealt = math.comb(len(deck), starting_hand_size)
print(f"{unique_hands_dealt=:,}")

# from the 6 dealt, you could discard any 2 of them
unique_sub_hands = math.comb(starting_hand_size, hand_size)
print(f"{unique_sub_hands=:,}")

# this means you can have any combination of 52c6 * 6c2 number of hands
total_possible_hands = unique_hands_dealt * unique_sub_hands
print(f"{total_possible_hands=:,}")

# crib will be made up of (52-6)c2 * 2 of your 6 cards (or 6c2)
possible_crib_hands = math.comb(len(deck) - starting_hand_size, 2) * unique_sub_hands
print(f"{possible_crib_hands=:,}")

unique_hands=12,994,800
unique_hands_dealt=20,358,520
unique_sub_hands=15
total_possible_hands=305,377,800
possible_crib_hands=15,525


In [8]:
# set of all possible 6 card starting hands
starting_hand_list = list(combinations(deck, r=starting_hand_size))
# proof of no duplicates for peace of mind
starting_hand_set = frozenset(starting_hand_list)

In [9]:
print(f"{len(starting_hand_list)=:,}")
print(f"{starting_hand_list[:10]}")

len(starting_hand_list)=20,358,520
[('2H', '3H', '4H', '5H', '6H', '7H'), ('2H', '3H', '4H', '5H', '6H', '8H'), ('2H', '3H', '4H', '5H', '6H', '9H'), ('2H', '3H', '4H', '5H', '6H', 'TH'), ('2H', '3H', '4H', '5H', '6H', 'JH'), ('2H', '3H', '4H', '5H', '6H', 'QH'), ('2H', '3H', '4H', '5H', '6H', 'KH'), ('2H', '3H', '4H', '5H', '6H', 'AH'), ('2H', '3H', '4H', '5H', '6H', '2D'), ('2H', '3H', '4H', '5H', '6H', '3D')]


In [10]:
points_df = pd.read_csv("crib_hands.csv")

# making columns easier to work with
points_df["hand"] = (
    points_df["hand_card_1"]
    + ","
    + points_df["hand_card_2"]
    + ","
    + points_df["hand_card_3"]
    + ","
    + points_df["hand_card_4"]
    + ","
    + points_df["cut_card"]
)
points_df = points_df.drop(
    columns=["hand_card_1", "hand_card_2", "hand_card_3", "hand_card_4", "cut_card"]
)

points_df.set_index("hand", inplace=True)

points_dict = points_df.T.to_dict(
    orient="list",
)

In [11]:
class PersonalHand:
    def __init__(self, cards: tuple[str]):
        self.cards = cards
        self.cards_str = ",".join(cards)
        self.cards_set = set(cards)


class PlayedHand:
    def __init__(self, hand_cards: PersonalHand, cut_card: str):
        self.hand_cards = hand_cards
        self.cut_card = cut_card
        self.points: int = self._get_points(hand_cards, cut_card)

    def _get_points(self, hand_cards: PersonalHand, cut_card: str):
        # TODO
        # val = points_df.loc[
        #     (points_df["hand"] == hand_cards.cards_str)
        #     & (points_df["cut_card"] == cut_card)
        # ].points.values[0]

        hand = hand_cards.cards_str + "," + cut_card

        try:
            val = points_dict[hand]
        except KeyError:
            print(hand)
            raise

        return int(val[0])


class PossibleHand:
    def __init__(self, four_card_hand: PersonalHand, possible_crib_cards: tuple[str]):
        self.hand = four_card_hand
        self.possible_crib_cards = possible_crib_cards

        self.possible_played_hands: list[PlayedHand] = []
        self.possible_points: list[int] = []
        self.points_max: int = 0
        self.points_min: int = 29
        for cut_card in possible_crib_cards:
            played_hand = PlayedHand(four_card_hand, cut_card)
            self.possible_played_hands.append(played_hand)
            self.possible_points.append(played_hand.points)

            if played_hand.points > self.points_max:
                self.points_max = played_hand.points
            if played_hand.points < self.points_min:
                self.points_min = played_hand.points

        # self.possible_points = [hand.points for hand in self.possible_played_hands]

        self.points_ev = statistics.mean(self.possible_points)
        self.points_stdev = statistics.stdev(self.possible_points)
        self.points_median = statistics.median(self.possible_points)
        # self.points_max = max(self.possible_points)
        # self.points_min = min(self.possible_points)


class DealtHand:
    def __init__(self, cards: tuple[str]):
        self.cards = cards

        _copied_deck = deck.copy()

        for card in cards:
            _copied_deck.remove(card)

        self.possible_crib_cards = _copied_deck
        _possible_hands_tuples = list(combinations(cards, r=hand_size))
        self.possible_personal_hands = [
            PersonalHand(hand) for hand in _possible_hands_tuples
        ]

        self.possible_hands = [
            PossibleHand(hand, self.possible_crib_cards)
            for hand in self.possible_personal_hands
        ]

        self.ev_hand = max(self.possible_hands, key=lambda hand: hand.points_ev)
        self.max_hand = max(self.possible_hands, key=lambda hand: hand.points_max)
        self.median_hand = max(self.possible_hands, key=lambda hand: hand.points_median)

In [12]:
columns = [
    "dealt_hand",
    "max_hand",
    "max_hand_ev_points",
    "max_hand_max_points",
    "max_hand_median_points",
    "max_hand_stdev_points",
    "median_hand",
    "median_hand_ev_points",
    "median_hand_max_points",
    "median_hand_median_points",
    "median_hand_stdev_points",
    "ev_hand",
    "ev_hand_ev_points",
    "ev_hand_max_points",
    "ev_hand_median_points",
    "ev_hand_stdev_points",
]

df = pd.DataFrame(columns=columns)

In [13]:
from tqdm import tqdm

with open("crib_hands_output.csv", "w") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(columns)

    for starting_hand in tqdm(starting_hand_list):
        staring_dealt_hand: DealtHand = DealtHand(starting_hand)

        row = [
            ",".join(staring_dealt_hand.cards),
        ]

        best_hands = [
            staring_dealt_hand.max_hand,
            staring_dealt_hand.median_hand,
            staring_dealt_hand.ev_hand,
        ]

        for best_hand in best_hands:
            row.append(best_hand.hand.cards_str)
            row.append(best_hand.points_ev)
            row.append(best_hand.points_max)
            row.append(best_hand.points_median)
            row.append(best_hand.points_stdev)

        writer.writerow(row)

        # df = pd.concat([pd.DataFrame([[1,2]], columns=df.columns), df], ignore_index=True)
        # df = pd.concat([pd.DataFrame([row], columns=df.columns), df], ignore_index=True)

100%|██████████| 20358520/20358520 [19:22:33<00:00, 291.86it/s]    
